# LoRA Fine-Tuning Experiments: Systematic Comparison on IMDB Sentiment Analysis

**Objective:**
Compare parameter-efficient fine-tuning strategies for DistilBERT on IMDB movie reviews, demonstrating how LoRA achieves near-full-fine-tuning performance with <2% of trainable parameters.

**Experimental Design:**
1. **Baseline (Classification Head Only)** - Train only the final layer (1.5K params)
2. **Full Fine-Tuning** - Upper bound comparison (67M params)
3. **LoRA Configurations** - Systematically vary rank (r=4, 8, 16) and target modules (Q+V, All Attention, All Attention+FFN)


**Key Questions:**
- Does rank matter? (r=4 vs r=8 vs r=16)
- Which matrices should we target? (Q+V vs All Attention)
- Can LoRA match full fine-tuning performance?
- What's the parameter efficiency tradeoff?

**Why DistilBERT?**
- 67M parameters (40% smaller than BERT-base)
- Fast iteration for experiments (~3-5 min per config on A100)
- Well-established baseline for sentiment classification
- Widely used in production systems

**Tech Stack:**
- `transformers` - HuggingFace DistilBERT implementation
- `datasets` - IMDB dataset (25K train, 25K test)
- `peft` - LoRA implementation via HuggingFace PEFT
- `torch` - PyTorch backend with fp16 mixed precision

**Hardware:**
- Google Colab Pro with NVIDIA A100 GPU (40GB)
- Training time: ~40 minutes total for all 7 configurations

**Expected Results:**
- Baseline: ~82% accuracy
- Full Fine-Tuning: ~93% accuracy with 100% trainable params
- LoRA (Q+V r=8): ~92% accuracy with 1.1% trainable params
- LoRA (All Attention+FFN r=8): ~93% accuracy with 1.88% trainable params



## Setup & Installation

In [ ]:
# Install required packages
!pip install -q transformers datasets peft accelerate evaluate scikit-learn

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datetime import datetime
import time

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cpu":
    print("⚠️  Training on CPU will be slower. Consider using Google Colab with free GPU.")

Using device: cuda


---
## Part 1: Load and Prepare IMDB Dataset

IMDB is a binary sentiment classification dataset:
- **Negative** (0): Bad movie reviews
- **Positive** (1): Good movie reviews
- 25,000 training samples
- 25,000 test samples

In [ ]:
# Load IMDB dataset
print("Loading IMDB dataset...")
imdb = load_dataset("imdb")

print("\n=== Dataset Info ===")
print(f"Splits: {list(imdb.keys())}")
print(f"Train size: {len(imdb['train'])} samples")
print(f"Test size: {len(imdb['test'])} samples")

print("\n=== Example Review ===")
example = imdb['train'][0]
print(f"Text: {example['text'][:200]}...")
print(f"Label: {example['label']} ({'Positive' if example['label'] == 1 else 'Negative'})")

Loading IMDB dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]


=== Dataset Info ===
Splits: ['train', 'test', 'unsupervised']
Train size: 25000 samples
Test size: 25000 samples

=== Example Review ===
Text: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ev...
Label: 0 (Negative)


In [ ]:
# Load DistilBERT tokenizer
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

print(f"Loaded tokenizer: {model_name}")
print(f"Vocab size: {tokenizer.vocab_size:,}")

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Loaded tokenizer: distilbert-base-uncased
Vocab size: 30,522


In [ ]:
# Tokenization function
def tokenize_function(examples):
    """
    Tokenize IMDB reviews with padding and truncation.
    Max length 512 tokens (IMDB reviews can be long).
    """
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=512
    )

# Tokenize datasets (batched for efficiency)
print("Tokenizing dataset... (this may take 1-2 minutes)")
start_time = time.time()
imdb_tokenized = imdb.map(tokenize_function, batched=True)
print(f"Tokenization complete in {time.time() - start_time:.1f} seconds")

# Set format for PyTorch
imdb_tokenized.set_format(
    type='torch',
    columns=['input_ids', 'attention_mask', 'label']
)

Tokenizing dataset... (this may take 1-2 minutes)


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Tokenization complete in 42.2 seconds


In [ ]:
# Create smaller subsets for faster experimentation (optional)
# Comment out these lines to use full dataset

USE_SUBSET = False  # Set to False for full dataset

if USE_SUBSET:
    train_dataset = imdb_tokenized['train'].shuffle(seed=42).select(range(10000))
    test_dataset = imdb_tokenized['test'].shuffle(seed=42).select(range(2000))
    print("📊 Using subset for faster training:")
    print(f"  Train: {len(train_dataset)} samples")
    print(f"  Test: {len(test_dataset)} samples")
else:
    train_dataset = imdb_tokenized['train']
    test_dataset = imdb_tokenized['test']
    print("📊 Using full dataset:")
    print(f"  Train: {len(train_dataset)} samples")
    print(f"  Test: {len(test_dataset)} samples")

📊 Using full dataset:
  Train: 25000 samples
  Test: 25000 samples


In [ ]:
# Check label balance
from collections import Counter

train_labels = [int(x) for x in train_dataset['label']]
test_labels = [int(x) for x in test_dataset['label']]

train_dist = Counter(train_labels)
test_dist = Counter(test_labels)

print(f"Train - Negative: {train_dist[0]}, Positive: {train_dist[1]}")
print(f"Test  - Negative: {test_dist[0]}, Positive: {test_dist[1]}")

Train - Negative: 12500, Positive: 12500
Test  - Negative: 12500, Positive: 12500


---
## Part 2: Define Evaluation Metrics

In [ ]:
def compute_metrics(eval_pred):
    """
    Calculate accuracy, precision, recall, and F1 score.
    Called automatically by Trainer during evaluation.
    """
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='binary'
    )

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

## Approach 1 -  Classification Head Only

**Strategy:**
- Freeze all DistilBERT transformer layers (66.95M parameters)
- Train only the final classification head (1,538 parameters)
- Establishes lower bound for comparison

**Why This Baseline:**
- Shows performance with minimal adaptation
- Demonstrates need for fine-tuning transformer layers
- Common starting point in low-resource settings

**Expected Performance:**
- Fast training (~2 minutes on A100)
- Accuracy: ~82% on IMDB
- ~10% lower than LoRA configurations

**Configuration:**
- Trainable params: 1,538 (0.002% of total)
- Learning rate: 1e-3 (higher than full fine-tuning)
- Epochs: 3
- Batch size: 16

In [ ]:
results_list = []

In [ ]:
# Load pretrained DistilBERT for binary classification
model_last_layer = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2  # Binary: negative (0) or positive (1)
)

# Freeze all parameters
for param in model_last_layer.parameters():
    param.requires_grad = False

# Unfreeze only the classification head
for param in model_last_layer.classifier.parameters():
    param.requires_grad = True

# Count parameters
trainable_params = sum(p.numel() for p in model_last_layer.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model_last_layer.parameters())

print("=== Last Layer Fine-Tuning ===")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Trainable %: {100 * trainable_params / total_params:.2f}%")

Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
classifier.bias         | MISSING    | 
classifier.weight       | MISSING    | 
pre_classifier.weight   | MISSING    | 
pre_classifier.bias     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


=== Last Layer Fine-Tuning ===
Total parameters: 66,955,010
Trainable parameters: 1,538
Trainable %: 0.00%


In [ ]:
# Training configuration for last layer
training_args_last_layer = TrainingArguments(
    output_dir="./results/baseline",
    num_train_epochs=3,
    per_device_train_batch_size=16,   # Reduce to 8 if out of memory
    per_device_eval_batch_size=32,
    learning_rate=1e-3,            # A higher LR workfor classification head only
    weight_decay=0.01,
    warmup_steps=100,
    logging_steps=500,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    report_to="none",
)

# Initialize Trainer
trainer_last_layer = Trainer(
    model=model_last_layer,
    args=training_args_last_layer,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

print("Trainer initialized for last layer fine-tuning")

Trainer initialized for last layer fine-tuning


In [ ]:
# Train last layer only
print("\n🚀 Starting last layer fine-tuning...\n")
start_time = datetime.now()
trainer_last_layer.train()
training_time_seconds = (datetime.now() - start_time).total_seconds()

print(f"\n✅ Training complete in {training_time_seconds:.1f} seconds ({training_time_seconds/60:.1f} minutes)")

# Store in minutes for the results table
training_time_last_layer = training_time_seconds / 60



🚀 Starting last layer fine-tuning...



Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.517262,0.474561,0.807520,0.858783,0.736080,0.792711
2,0.486382,0.441436,0.817840,0.856004,0.764240,0.807523
3,0.468496,0.433481,0.821760,0.833499,0.804160,0.818567


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias'].
There were unexpected keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.beta', 'distilbert.embeddings.LayerNorm.gamma'].



✅ Training complete in 108.8 seconds (1.8 minutes)


In [ ]:
# Evaluate last layer model
print("\n📊 Evaluating last layer model...")
last_layer_results = trainer_last_layer.evaluate()


print("\n=== Last Layer Results ===")
for key, value in last_layer_results.items():
    if not key.startswith('eval_runtime'):
        print(f"{key}: {value:.4f}")



📊 Evaluating last layer model...



=== Last Layer Results ===
eval_loss: 0.4335
eval_accuracy: 0.8218
eval_precision: 0.8335
eval_recall: 0.8042
eval_f1: 0.8186
eval_samples_per_second: 1801.8010
eval_steps_per_second: 56.3600
epoch: 3.0000


In [ ]:
results_list.append({
    'Configuration': 'Baseline (head only)',
    'Rank': '-',
    'Target Modules': 'None',
    'Trainable Params': trainable_params,
    'Trainable %': 100 * trainable_params / total_params,
    'Accuracy': last_layer_results['eval_accuracy'],
    'F1 Score': last_layer_results.get('eval_f1', 0),
    'Loss': last_layer_results['eval_loss'],
    'Training Time (sec)':training_time_seconds,
    'Training Time (min)': training_time_last_layer  # Already in minutes
})

In [ ]:
results_list

[{'Configuration': 'Baseline (head only)',
  'Rank': '-',
  'Target Modules': 'None',
  'Trainable Params': 1538,
  'Trainable %': 0.0022970648499641773,
  'Accuracy': 0.82176,
  'F1 Score': 0.8185667752442997,
  'Loss': 0.43348124623298645,
  'Training Time (sec)': 108.774253,
  'Training Time (min)': 1.8129042166666667}]

In [ ]:
#delete!
import pandas as pd

results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values('Trainable Params')

# Format for display
display_df = results_df.copy()
display_df['Trainable Params'] = display_df['Trainable Params'].apply(lambda x: f"{x:,}")
display_df['Trainable %'] = display_df['Trainable %'].apply(lambda x: f"{x:.2f}%")
display_df['Accuracy'] = display_df['Accuracy'].apply(lambda x: f"{x:.4f}")
display_df['F1 Score'] = display_df['F1 Score'].apply(lambda x: f"{x:.4f}")
display_df['Loss'] = display_df['Loss'].apply(lambda x: f"{x:.4f}")
display_df['Training Time (min)'] = display_df['Training Time (min)'].apply(lambda x: f"{x:.12f}")

print("\n" + "="*90)
print("FINAL RESULTS - ALL EXPERIMENTS")
print("="*90)
print(display_df.to_string(index=False))
print("="*90)



FINAL RESULTS - ALL EXPERIMENTS
       Configuration Rank Target Modules Trainable Params Trainable % Accuracy F1 Score   Loss  Training Time (sec) Training Time (min)
Baseline (head only)    -           None            1,538       0.00%   0.8100   0.8079 0.5030            43.942147      0.732369116667
    Full Fine-Tuning    -            All       66,955,010     100.00%   0.9225   0.9217 0.2955           311.860436      5.197673933333


## Approach 2 - Full Fine-Tuning

**Strategy:**
- Train all 66,955,010 parameters (100% of model)
- No frozen layers - complete adaptation to IMDB
- Establishes upper bound for comparison

**Why This Baseline:**
- Shows maximum achievable performance with unlimited parameters
- Tests whether LoRA can match unrestricted fine-tuning
- Standard approach before parameter-efficient methods

**Expected Performance:**
- Slower training (~10-12 minutes on A100)
- Accuracy: ~93.3% on IMDB
- Best case scenario - what we're trying to match with <2% params

**Configuration:**
- Trainable params: 66,955,010 (100%)
- Learning rate: 2e-5 (lower than LoRA to prevent catastrophic forgetting)
- Epochs: 3
- Batch size: 16


In [ ]:
# After your baseline, before LoRA loop
print("\n" + "="*70)
print("Training: Full Fine-Tuning (All Parameters)")
print("="*70)

# Load model (don't freeze anything)
model_full = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

# All params trainable by default
trainable_params = sum(p.numel() for p in model_full.parameters() if p.requires_grad)
print(f"Trainable params: {trainable_params:,} (100%)")


training_args_full = TrainingArguments(
    output_dir="./results/full_finetuning",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,           # LR
    weight_decay=0.01,
    warmup_steps=100,
    logging_steps=500,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    report_to="none",
)

trainer_full = Trainer(
    model=model_full,
    args=training_args_full,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

start_time = datetime.now()
trainer_full.train()
training_time_full_seconds = (datetime.now() - start_time).total_seconds()
training_time_full = training_time_full_seconds / 60


full_results = trainer_full.evaluate()

# Store results
results_list.append({
    'Configuration': 'Full Fine-Tuning',
    'Rank': '-',
    'Target Modules': 'All',
    'Trainable Params': trainable_params,
    'Trainable %': 100.0,
    'Accuracy': full_results['eval_accuracy'],
    'F1 Score': full_results.get('eval_f1', 0),
    'Loss': full_results['eval_loss'],
    'Training Time (sec)':training_time_full_seconds,
    'Training Time (min)': training_time_full
})

print(f"✅ Full fine-tuning complete: {full_results['eval_accuracy']:.4f}")

del model_full, trainer_full
torch.cuda.empty_cache()


Training: Full Fine-Tuning (All Parameters)


Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
classifier.bias         | MISSING    | 
classifier.weight       | MISSING    | 
pre_classifier.weight   | MISSING    | 
pre_classifier.bias     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Trainable params: 66,955,010 (100%)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.227627,0.217665,0.913320,0.958064,0.864480,0.908869
2,0.153301,0.237071,0.930640,0.923125,0.939520,0.931250
3,0.093170,0.275410,0.932640,0.929547,0.936240,0.932882


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias'].
There were unexpected keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.beta', 'distilbert.embeddings.LayerNorm.gamma'].


✅ Full fine-tuning complete: 0.9326


##  Approach 3 - LoRA Experiments

**What is LoRA?**

LoRA (Low-Rank Adaptation) decomposes weight updates into two smaller matrices:
```
ΔW = B × A
W_new = W_frozen + (α/r) × B × A
```

Where:
- **W_frozen:** Original pretrained weights (remain frozen)
- **B:** Matrix of shape (m, r) - trainable, initialized to zero
- **A:** Matrix of shape (r, n) - trainable, initialized with Gaussian noise
- **r:** Rank (we test r=4, 8, 16)
- **α:** Scaling factor (set to 2r in our experiments)

**Why This Works:**

Weight updates during fine-tuning have low intrinsic rank - the task-specific changes occupy a surprisingly small subspace. Instead of updating all 589,824 parameters in a 768×768 attention matrix, LoRA uses just 12,288 parameters (r=8) to achieve comparable results.

**Our Experiment Design:**

We systematically vary two dimensions:

1. **Rank (r):** 4, 8, 16 - does higher rank improve performance?
2. **Target Modules:** 
   - Q+V only (paper recommendation)
   - All Attention (Q, K, V, O)
   - All Attention + FFN (maximum adaptation)

**Configurations Tested:**
- Q+V (r=4): 666K params (0.99%)
- Q+V (r=8): 740K params (1.10%)
- Q+V (r=16): 887K params (1.32%)
- All Attention (r=8): 887K params (1.32%)
- All Attention+FFN (r=8): 1.26M params (1.88%)

**Key Questions:**
- Does rank matter? (diminishing returns hypothesis)
- Is Q+V sufficient, or do we need all attention matrices?
- Can adding FFN layers close the gap to full fine-tuning?

**Expected Results:**
- All configs should significantly outperform baseline (~82%)
- Higher ranks should show diminishing returns
- All Attention+FFN should approach full fine-tuning (~93%)



## Single LoRA Configuration Example

Before running the full experiment loop, let's walk through a single LoRA configuration step-by-step to understand the implementation.

**This example:** Q+V matrices with r=8
- Trainable params: ~740K (1.10% of model)
- Expected accuracy: ~92.5%

**After understanding this single run, we'll automate the process to test all 5 LoRA configurations systematically.**


In [ ]:
# Load fresh DistilBERT model for LoRA
model_lora = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

# Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,       # Sequence classification
    r=8,                               # Rank of decomposition (typical: 4-16)
    lora_alpha=16,                     # Scaling factor (often 2*r)
    lora_dropout=0.1,                  # Dropout for regularization
    target_modules=["q_lin", "v_lin" ], # Apply LoRA to query & value in attention
    bias="none",                       # Don't train bias terms
)

# Apply LoRA to model
model_lora = get_peft_model(model_lora, lora_config)

print("=== LoRA Configuration ===")
model_lora.print_trainable_parameters()
print(f"\nLoRA rank (r): {lora_config.r}")
print(f"Target modules: {lora_config.target_modules}")

Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_transform.bias    | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
classifier.bias         | MISSING    | 
classifier.weight       | MISSING    | 
pre_classifier.bias     | MISSING    | 
pre_classifier.weight   | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


=== LoRA Configuration ===
trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925

LoRA rank (r): 8
Target modules: {'q_lin', 'v_lin'}


In [ ]:
# Training configuration for LoRA
training_args_lora = TrainingArguments(
    output_dir="./distilbert_imdb_lora",
    num_train_epochs=3,
    per_device_train_batch_size=16,   # Reduce to 8 if out of memory
    per_device_eval_batch_size=32,
    learning_rate=3e-4,                # Higher LR typical for LoRA (1e-4 to 5e-4)
    weight_decay=0.01,
    warmup_steps=100,
    logging_steps=100,
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=torch.cuda.is_available(),
    report_to="none",
)

# Initialize Trainer
trainer_lora = Trainer(
    model=model_lora,
    args=training_args_lora,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

print("Trainer initialized for LoRA fine-tuning")

Trainer initialized for LoRA fine-tuning


In [ ]:
# Train with LoRA
print("\n🚀 Starting LoRA fine-tuning...\n")
start_time = time.time()

trainer_lora.train()

training_time_lora = time.time() - start_time
print(f"\n✅ Training complete in {training_time_lora:.1f} seconds ({training_time_lora/60:.1f} minutes)")


🚀 Starting LoRA fine-tuning...



Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
200,0.315263,0.308032,0.862000,0.844488,0.879098,0.861446
400,0.243535,0.341206,0.884000,0.890756,0.868852,0.879668
600,0.250108,0.275920,0.892000,0.918502,0.854508,0.885350
800,0.201878,0.282307,0.893000,0.892784,0.887295,0.890031



✅ Training complete in 38.3 seconds (0.6 minutes)


In [ ]:
# Evaluate LoRA model
print("\n📊 Evaluating LoRA model...")
lora_results = trainer_lora.evaluate()

print("\n=== LoRA Results ===")
for key, value in lora_results.items():
    if not key.startswith('eval_runtime'):
        print(f"{key}: {value:.4f}")


📊 Evaluating LoRA model...



=== LoRA Results ===
eval_loss: 0.2823
eval_accuracy: 0.8930
eval_precision: 0.8928
eval_recall: 0.8873
eval_f1: 0.8900
eval_samples_per_second: 1404.2380
eval_steps_per_second: 44.9360
epoch: 3.0000


In [ ]:
# Save LoRA adapter
model_lora.save_pretrained("./distilbert_imdb_lora_adapter")
print("\n💾 LoRA adapter saved to ./distilbert_imdb_lora_adapter")
print("Note: This only saves the adapter weights (~few MB), not the full model!")


💾 LoRA adapter saved to ./distilbert_imdb_lora_adapter
Note: This only saves the adapter weights (~few MB), not the full model!


## For Loop LoRA

To efficiently compare different LoRA configurations, we'll run all experiments in a single loop. This ensures consistent training conditions and makes results directly comparable.

**Loop Structure:**
- Tests 5 LoRA configurations with varying rank and target modules
- Uses identical hyperparameters (learning rate, epochs, batch size)
- Tracks results in a structured format for easy comparison
- Cleans up GPU memory between runs

**Configurations:**
1. Q+V (r=4) - Minimal parameters
2. Q+V (r=8) - Sweet spot candidate
3. Q+V (r=16) - Diminishing returns test
4. All Attention (r=8) - Does adding K and O help?
5. All Attention+FFN (r=8) - Maximum adaptation

**Runtime:** ~15-20 minutes total on A100 (3-4 min per config)


In [ ]:
#lora_configs = [
#    {"name": "Q+V r=4", "modules": ["q_lin", "v_lin"], "rank": 4, "alpha": 8}]

In [ ]:
lora_configs = [
    {"name": "Q+V r=4", "modules": ["q_lin", "v_lin"], "rank": 4},
    {"name": "Q+V r=8", "modules": ["q_lin", "v_lin"], "rank": 8},
    {"name": "Q+V r=16", "modules": ["q_lin", "v_lin"], "rank": 16},
    {"name": "All Attention r=8", "modules": ["q_lin", "k_lin", "v_lin", "out_lin"], "rank": 8},
    {"name": "All Attention+FFN r=8", "modules": ["q_lin", "k_lin", "v_lin", "out_lin", "ffn.lin1", "ffn.lin2"], "rank": 8},
]

In [ ]:
#Training arguments (same for all LoRA configs)
training_args_lora = TrainingArguments(
    output_dir="./results/lora",  # Will be updated per config in loop
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=3e-4,           # Only difference
    weight_decay=0.01,
    warmup_steps=100,
    logging_steps=500,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    report_to="none",
)

In [ ]:

# Loop through each LoRA configuration
for config in lora_configs:
    print(f"\n{'='*70}")
    print(f"🚀 Training: {config['name']}")
    print(f"{'='*70}\n")

    # Load fresh model for each config
    model_lora = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2
    )

    # Configure LoRA
    lora_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=config['rank'],
        lora_alpha=config['rank']*2,
        lora_dropout=0.1,
        target_modules=config['modules'],
        bias="none",
    )

    # Apply LoRA to model
    model_lora = get_peft_model(model_lora, lora_config)

    # Print trainable parameters
    print("=== LoRA Configuration ===")
    model_lora.print_trainable_parameters()

    # Count parameters
    trainable_params = sum(p.numel() for p in model_lora.parameters() if p.requires_grad)
    print(f"\nTotal parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Trainable %: {100 * trainable_params / total_params:.2f}%")

    # Update output dir for this config
    training_args_lora.output_dir = f"./results/{config['name'].replace(' ', '_')}"

    # Initialize Trainer
    trainer_lora = Trainer(
        model=model_lora,
        args=training_args_lora,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics,
    )

    print(f"\n🚀 Starting training for {config['name']}...\n")

    # Train and time it
    start_time = datetime.now()
    trainer_lora.train()
    training_time_seconds = (datetime.now() - start_time).total_seconds()
    training_time_lora = training_time_seconds / 60  # Convert to minutes

    print(f"\n✅ Training complete in {training_time_seconds:.1f} seconds ({training_time_lora:.1f} minutes)")

    # Evaluate
    print(f"\n📊 Evaluating {config['name']}...")
    lora_results = trainer_lora.evaluate()

    print(f"\n=== {config['name']} Results ===")
    for key, value in lora_results.items():
        if not key.startswith('eval_runtime'):
            print(f"{key}: {value:.4f}")

    # Store results in list
    results_list.append({
        'Configuration': config['name'],
        'Rank': config['rank'],
        'Target Modules': ', '.join(config['modules']),
        'Trainable Params': trainable_params,
        'Trainable %': 100 * trainable_params / total_params,
        'Accuracy': lora_results['eval_accuracy'],
        'F1 Score': lora_results.get('eval_f1', 0),
        'Loss': lora_results['eval_loss'],
        'Training Time (sec)': training_time_seconds,
        'Training Time (min)': training_time_lora
    })

    print(f"\n✅ {config['name']} results stored")

    # Clean up memory
    del model_lora, trainer_lora
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    print(f"\n{'='*70}")
    print(f"✅ {config['name']} COMPLETE")
    print(f"{'='*70}\n")

print("\n" + "🎉"*35)
print("ALL LORA EXPERIMENTS COMPLETE")
print("🎉"*35 + "\n")


🚀 Training: Q+V r=4



Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
classifier.bias         | MISSING    | 
classifier.weight       | MISSING    | 
pre_classifier.weight   | MISSING    | 
pre_classifier.bias     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


=== LoRA Configuration ===
trainable params: 665,858 || all params: 67,620,868 || trainable%: 0.9847

Total parameters: 66,955,010
Trainable parameters: 665,858
Trainable %: 0.99%

🚀 Starting training for Q+V r=4...



Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.246019,0.229318,0.909760,0.936658,0.878960,0.906892
2,0.219671,0.207073,0.919680,0.939363,0.897280,0.917840
3,0.197177,0.203319,0.922840,0.920854,0.925200,0.923022


...

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
classifier.bias         | MISSING    | 
classifier.weight       | MISSING    | 
pre_classifier.weight   | MISSING    | 
pre_classifier.bias     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


=== LoRA Configuration ===
trainable params: 1,255,682 || all params: 68,210,692 || trainable%: 1.8409

Total parameters: 66,955,010
Trainable parameters: 1,255,682
Trainable %: 1.88%

🚀 Starting training for All Attention+FFN r=8...



Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.226368,0.200619,0.921920,0.942301,0.898880,0.920079
2,0.184259,0.185540,0.929920,0.947088,0.910720,0.928548
3,0.140397,0.194711,0.933240,0.925914,0.941840,0.933809



✅ Training complete in 329.8 seconds (5.5 minutes)

📊 Evaluating All Attention+FFN r=8...



=== All Attention+FFN r=8 Results ===
eval_loss: 0.1947
eval_accuracy: 0.9332
eval_precision: 0.9259
eval_recall: 0.9418
eval_f1: 0.9338
eval_samples_per_second: 1026.2960
eval_steps_per_second: 32.1030
epoch: 3.0000

✅ All Attention+FFN r=8 results stored

✅ All Attention+FFN r=8 COMPLETE


🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉
ALL LORA EXPERIMENTS COMPLETE
🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉



## Part 5: Comparison & Analysis

In [ ]:
results_list

[{'Configuration': 'Baseline (head only)',
  'Rank': '-',
  'Target Modules': 'None',
  'Trainable Params': 1538,
  'Trainable %': 0.0022970648499641773,
  'Accuracy': 0.82176,
  'F1 Score': 0.8185667752442997,
  'Loss': 0.43348124623298645,
  'Training Time (sec)': 108.774253,
  'Training Time (min)': 1.8129042166666667},
 {'Configuration': 'Full Fine-Tuning',
  'Rank': '-',
  'Target Modules': 'All',
  'Trainable Params': 66955010,
  'Trainable %': 100.0,
  'Accuracy': 0.93264,
  'F1 Score': 0.9328816261458749,
  'Loss': 0.27540960907936096,
  'Training Time (sec)': 212.22895,
  'Training Time (min)': 3.5371491666666666},
 {'Configuration': 'Q+V r=4',
  'Rank': 4,
  'Target Modules': 'q_lin, v_lin',
  'Trainable Params': 665858,
  'Trainable %': 0.9944857001738929,
  'Accuracy': 0.92284,
  'F1 Score': 0.9230216688614868,
  'Loss': 0.20331937074661255,
  'Training Time (sec)': 212.706379,
  'Training Time (min)': 3.5451063166666668},
 {'Configuration': 'Q+V r=8',
  'Rank': 8,
  'Targe

In [ ]:
import pandas as pd
results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values('Trainable Params')

# Format for display
display_df = results_df.copy()
display_df['Trainable Params'] = display_df['Trainable Params'].apply(lambda x: f"{x:,}")
display_df['Trainable %'] = display_df['Trainable %'].apply(lambda x: f"{x:.2f}%")
display_df['Accuracy'] = display_df['Accuracy'].apply(lambda x: f"{x:.4f}")
display_df['F1 Score'] = display_df['F1 Score'].apply(lambda x: f"{x:.4f}")
display_df['Loss'] = display_df['Loss'].apply(lambda x: f"{x:.4f}")
display_df['Training Time (min)'] = display_df['Training Time (min)'].apply(lambda x: f"{x:.12f}")

print("\n" + "="*90)
print("FINAL RESULTS - ALL EXPERIMENTS")
print("="*90)
print(display_df.to_string(index=False))
print("="*90)



FINAL RESULTS - ALL EXPERIMENTS
        Configuration Rank                                   Target Modules Trainable Params Trainable % Accuracy F1 Score   Loss  Training Time (sec) Training Time (min)
 Baseline (head only)    -                                             None            1,538       0.00%   0.8218   0.8186 0.4335           108.774253      1.812904216667
              Q+V r=4    4                                     q_lin, v_lin          665,858       0.99%   0.9228   0.9230 0.2033           212.706379      3.545106316667
              Q+V r=8    8                                     q_lin, v_lin          739,586       1.10%   0.9248   0.9250 0.2017           212.490001      3.541500016667
             Q+V r=16   16                                     q_lin, v_lin          887,042       1.32%   0.9267   0.9270 0.1988           212.451277      3.540854616667
    All Attention r=8    8                     q_lin, k_lin, v_lin, out_lin          887,042       1.32%   0.930

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

True
NVIDIA A100-SXM4-40GB
